## Packaging Starters
Starters are configured through the [traitlets](https://github.com/ipython/traitlets) system. As of `notebook 5.3`, these configurations can be [packaged as simple files](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Distributing%20Jupyter%20Extensions%20as%20Python%20Packages.html#Automatically-enabling-a-server-extension-and-nbextension). Here's a quick example, assuming the following file structure:

```yaml
my-project:
  - setup.py
  - MANIFEST.in
  # other good stuff like LICENSE, README.md, CHANGELOG.md, CODE_OF_CONDUCT.md
  - src:
    - my_project:
      - __init__.py
      - etc:
        - my-project-starter.json
```

### `MANIFEST.in`


```ini
# the usual suspects
include LICENSE README.md CHANGELOG.md CODE_OF_CONDUCT.md
# ensure the starter is included in the source distribution
recursive-include src *.json
```

### `setup.py`

```python
import setuptools

setuptools.setup(
    ...
    include_package_data=True,
    data_files=[
        (
            "etc/jupyter/jupyter_notebook_config.d",
            ["src/my_project/etc/my-project-starter.json"],
        )
    ]
    zip_safe=False
)
```